# Plot FVCOM input and output files
**Author: Jun Sasaki Coded: 2024-12-26 Updated: 2025-04-23**<br>
Plot and check FVCOM input & output netCDF and text files using xfvcom with matplotlib.
## Note
- contourf uses the value at the center of each mesh. Thus, at the boundaries (surface, bottom, lateral), the area between the edge and the center at each boundary mesh cannot be filled with color. To improve this, pcolormesh should be used.
- Bottom meshes become staircases. To avoid this, sigma coordinate transformation should be used.  

In [ ]:
import xarray as xr
import os
from xfvcom import FvcomDataLoader, FvcomAnalyzer, FvcomPlotConfig, FvcomPlotter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from math import ceil

from IPython.core.magic import register_cell_magic
@register_cell_magic
def skip(line, cell):
    print("This cell is skipped.")

png_dir = "PNG"
os.makedirs(png_dir, exist_ok=True)

# Plot output netcdf

In [ ]:
base_path = "~/Github/TB-FVCOM/goto2023/output"
base_path = os.path.expanduser(base_path)
ncfiles = ["TokyoBay18_r16_crossed_0001.nc"]
idx_ncfiles=0
ncfile_path = f"{base_path}/{ncfiles[idx_ncfiles]}"
ofc = FvcomDataLoader(ncfile_path=ncfile_path, time_tolerance=5)
ofc.ds

## 1-D time series plot for `varname(time, idx, k)`
Specify `varname`, node/nele/nobc number `idx`, and siglay/siglev number `k` (optional).

In [ ]:
#%%skip
varname = 'salinity'
# start=None; end=None
xlim=('2020-01-05',None)
ylim=None #(None, None)
idx=1  # node or nele number
k=0 # siglay or siglev number
pngfile = os.path.join(png_dir, f"{varname}_{xlim[0]}_{xlim[1]}")

plot_config = FvcomPlotConfig(width=8, height=2)
plotter = FvcomPlotter(ofc.ds, plot_config)
da = ofc.ds[varname]
fig, ax = plotter.ts_plot(da, index=idx, k=k, xlim=xlim, ylim=ylim)
fig.savefig(pngfile, dpi=300, bbox_inches='tight')
plt.show()
plt.close()

## 2-D time series plot for varname(time, k) in sigma coordinates
Vertical 2-D time series contourf at a specified node/nele in sigma coordinates.<br>
`fvcom` is an instance of `FvcomDataLoader`.
```Python
# (1) da with node/nele index=idx
da = fvcom.ds[varname]
fig, ax, cbar = plotter.ts_contourf(da, index=idx)
# (2) da sliced with node/nele index=idx
da = fvcom.ds[varname][:,:,idx]
fig, ax, cbar = plotter.ts_contourf(da)
```

In [ ]:
varname = 'salinity'
start=None; end=None
#start='2020-01-05'; end=None
xlim=(None, None)
#xlim=None
ylim=(-1, 0)
idx=1  # node or nele number
if xlim is None:
    pngfile = os.path.join(png_dir, f"{varname}_node-{idx}")
else:
    pngfile = os.path.join(png_dir, f"{varname}_node-{idx}_{xlim[0]}_{xlim[1]}")
plot_config = FvcomPlotConfig(width=8, height=2)
plotter = FvcomPlotter(ofc.ds, plot_config)
#da = ofc.ds[varname][:,:,idx]  # Set index=None
da = ofc.ds[varname]  # Set index=idx 
fig, ax, cbar = plotter.ts_contourf(da, index=idx, xlim=xlim, ylim=ylim)
fig.savefig(pngfile, dpi=300, bbox_inches='tight')
plt.show()
plt.close()

### Subplots

In [ ]:
idx=1
xlim=(None, None)
ylim=(-1,0)
pngfile = os.path.join(png_dir, f"subplots")
plot_config = FvcomPlotConfig()
plotter = FvcomPlotter(ofc.ds, plot_config)
fig, ax = plt.subplots(2,1, figsize=(8,4))
fig, ax[0], cbar = plotter.ts_contourf(ofc.ds['temp'], index=idx, xlim=xlim, ylim=ylim, ax=ax[0], title="")
fig, ax[1], cbar = plotter.ts_contourf(ofc.ds['salinity'], index=idx, xlim=xlim, ylim=ylim, ax=ax[1], title="")

## 2-D time series plot for varname(time, k) in z coordinates

In [ ]:
varname = 'temp'
idx=1
xlim=None
ylim=(None, 0.5)
plot_config = FvcomPlotConfig()
plotter = FvcomPlotter(ofc.ds, plot_config)
da = ofc.ds[varname] 
save_path = f"plot_{varname}.png"
surface_kwargs={"color":"black","linewidth":0.5}
ax=plotter.ts_contourf_z(da, index=idx, xlim=xlim, ylim=ylim, vmin=9.9, vmax=10.2, plot_surface=True, surface_kwargs=surface_kwargs)

## 2-D vertical section plot for varname(k, n) along a specified line in z coordinates
- Plot along a latitude (lat), longitude (lon), or polyline (line) with a horizontal spacing (m).
- Note 1: Vertical white lines (narrow rectangles) correspond to the areas between the neighboring sea and land mesh centers, which cannot be removed at present.
```Python
# line: A list of tuples [(lon0, lat0), (lon1, lat1), ...], creating a polyline
plotter.section_contourf_z(da, [lat=35.7, lon=139.7, line=line], spacing=50)
```

#### Along a specified latitude `lat`

In [ ]:
varname = 'temp'
plot_config = FvcomPlotConfig()
plotter = FvcomPlotter(ofc.ds, plot_config)
da = ofc.ds[varname].isel(time=20)
plotter.section_contourf_z(da, lat=35.6, ylim=(-20, 0), spacing=50)

#### Along a specified polyline `line`

In [ ]:
varname = 'temp'
line = [(140.062, 35.653), (139.72753, 35.3572), (139.74386, 35.31192),
        (139.799387, 35.241927), (139.721814, 35.096411)]
plot_config = FvcomPlotConfig()
plotter = FvcomPlotter(ofc.ds, plot_config)
da = ofc.ds[varname].isel(time=30)
plotter.section_contourf_z(da, line=line, spacing=100)

# Plot input netcdf files

#### Prepare netCDF files

In [ ]:
base_path = "~/Github/TB-FVCOM/goto_dye/input/input_steady/2020"
base_path = os.path.expanduser(base_path)
ncfiles = ["TokyoBay18_2020_wnd.nc","TokyoBay2020final_tsobc.nc", "TokyoBay2020julian_obc.nc",
           "TokyoBay2020kisarazufinal_sewer.nc", "TokyoBay2020final_river.nc",
           "TokyoBay2020final_sewer.nc"]

## Meteorological fields

#### Create Dataset for `*_wnd.nc` and retrieve its `varnames(time, node)` and `varnames(time, nele)`
```Python
varnames = ['uwind_speed', 'vwind_speed', 'air_temperature', 'cloud_cover', 'short_wave', 'long_wave',
            'relative_humidity', 'air_pressure', 'Precipitation']
```
#### Retrieve varnames automatically by specifying dims of (time, node) or (time, nele)
```Python
FvcomAnalyzer.get_variables_by_dims("time", "node")
FvcomAnalyzer.get_variables_by_dims("time", "nele")
```

In [ ]:
nc_met = os.path.join(base_path, ncfiles[0])
met = FvcomDataLoader(ncfile_path=nc_met, time_tolerance=5)
analyzer=FvcomAnalyzer(met.ds)
varnames_time_node = analyzer.get_variables_by_dims(("time", "node"))
varnames_time_nele = analyzer.get_variables_by_dims(("time", "nele"))
print(f"varnames(time, node)={varnames_time_node}")
print(f"varnames(time, nele)={varnames_time_nele}")
met.ds

### 1-D time series plot 

In [ ]:
varname = varnames_time_node[6]
#varname = varnames_time_nele[0]
print(f"Plot {varname}")
index = 0  # Specify a node/nele index. 
plot_config = FvcomPlotConfig()
plotter = FvcomPlotter(met.ds, plot_config)
da = met.ds[varname]
plotter.ts_plot(da, index=index)

### 1-D time series batch plots
Plot (time, node) variables and (time, nele) variables. One PNG contains `batch_size` subplots.

#### Batch plot for `varnames(time, node)`
Specify the number of panels in one figure by `batch_size` and the prefix of PNG file names by `png_prefix`. 
```Python
FvcomPlotter.ts_plot_in_batches(varnames, index, batch_size=batch_size, png_prefix=png_prefix)
```

In [ ]:
varnames = varnames_time_node
xlim=(None, None)
png_prefix = os.path.join(png_dir, "time_node")
plot_config = FvcomPlotConfig()
plotter = FvcomPlotter(met.ds, plot_config)

# Use ts_plot_in_batches in HelperMixin in helpers.py
plotter.ts_plot_in_batches(varnames=varnames, index=0, batch_size=5,
                           xlim=xlim, rolling_window=25, png_prefix=png_prefix)

#### Batch plot for `varnames(time, nele)`

In [ ]:
varnames = varnames_time_nele
xlim=(None, None)
png_prefix = os.path.join(png_dir, "time_nele")
plot_config = FvcomPlotConfig()
plotter = FvcomPlotter(met.ds, plot_config)

# Use ts_plot_in_batches in HelperMixin in helpers.py
plotter.ts_plot_in_batches(varnames=varnames, index=0, batch_size=5,
                           xlim=xlim, rolling_window=25, png_prefix=png_prefix)

### Wind vector time series plot
Plot time-series of vector components `(u, v)` and optionally their magnitude. Accept either DataArrays `(da_x, da_y)` or Dataset varnames `(varname_x, varname_y)` exclusively.
```Python
FvcomPlotter.ts_vector(da_x, da_y, index=0, rolling_window=25)
FvcomPlotter.ts_vector(varname_x=varname_x, varname_y=varname_y, index=0, rolling_window=25)
```

In [ ]:
png_file = os.path.join(png_dir, "wind_vector.png")
plot_config = FvcomPlotConfig()
plotter = FvcomPlotter(met.ds, plot_config)
da_x = met.ds.uwind_speed
da_y = met.ds.vwind_speed
fig, ax = plotter.ts_vector(da_x, da_y, index=0, rolling_window=25)
#fig, ax = plotter.ts_vector(varname_x='uwind_speed', varname_y='vwind_speed', index=0, rolling_window=25)
#ax.grid()
fig.savefig(png_file, dpi=300, bbox_inches='tight')

### River discharge time series plot

In [ ]:
nc_discharge = os.path.join(base_path, ncfiles[4])
discharge = FvcomDataLoader(ncfile_path=nc_discharge, time_tolerance=5)
plot_config = FvcomPlotConfig()
plotter = FvcomPlotter(discharge.ds, plot_config)
plotter.ts_discharge("river_flux", river_index=0, rolling_window=25, save_path='river_discharge.png')
#plotter.ts_discharge(varname="river_flux", river_index=0, start=start, end=end,
#                                  rolling_window=25, save_path="river_discharge.png"
#)

In [ ]:
plot_config = FvcomPlotConfig(figsize=(8, 2))
plotter = FvcomPlotter(fvcom.ds, plot_config)
var_name='river_temp'
# HelperMixin のメソッドを利用
plotter.plot_timeseries_for_river_in_batches(
    plotter=plotter,
    var_name=var_name,
    batch_size=5,
    start=start,
    end=end,
    rolling_window=25,
    save_prefix=var_name
)

## Example for input netcdf files

In [ ]:
base_path = "~/Github/TB-FVCOM/goto2023/input"
base_path = os.path.expanduser(base_path)
ncfiles = ["TokyoBay_tsobc.nc", "TokyoBay_julian_obc.nc","TokyoBayfinal_river.nc", "TokyoBayfinal16_river.nc",
           "TokyoBay18_z0.nc", "TokyoBay18_wnd.nc",
           "TokyoBay2020final_sewer.nc"]
index_ncfiles=5
ncfile_path = f"{base_path}/{ncfiles[index_ncfiles]}"
print(f"ncfile_path={ncfile_path}")
ifc = FvcomDataLoader(ncfile_path=ncfile_path, time_tolerance=5)
ifc.ds

## Compare with meteorological data
- Needs to install metdata; see [GitHub](https://github.com/jsasaki-utokyo/metdata)

In [ ]:
from metdata import gwo

datetime_ini = "2020-01-01 00:00:00"
datetime_end = "2021-01-01 00:00:00"
stn = "Chiba"
base_path = "/mnt/c/dat/met/JMA_DataBase/GWO/Hourly/"

met = gwo.Hourly(datetime_ini=datetime_ini, datetime_end=datetime_end, stn=stn, dirpath=base_path)
met.df.head()  # pandas.DataFrame

In [ ]:
met_ds = xr.Dataset.from_dataframe(met.df)
met_ds = met_ds.rename_dims({"index": "time"}).rename_vars({"index": "time"})
met_ds

In [ ]:
plot_config = FvcomPlotConfig(width=8, height=2)
plotter = FvcomPlotter(met_ds, plot_config)
start='2020-01-01'; end='2020-01-31'
plotter.plot_wind_vector_timeseries(u_var='u', v_var='v', nele=None, start=start, end=end, rolling_window=1,
                                    save_path="wind_vector_plot.png")

In [ ]:
plot_config = FvcomPlotConfig(width=8, height=2)
plotter = FvcomPlotter(ifc.ds, plot_config)
# ノード0の風速ベクトル時系列を2020年1月1日から12月31日までプロット
plotter.plot_wind_vector_timeseries(nele=0, start=start, end=end, rolling_window=1,
                                    save_path="wind_vector_plot.png")